# Suivi de Trajectoire

In [1]:
#pip install pygame

In [2]:
import math
import matplotlib.pyplot as plt
import pygame
import random
import sys
import numpy as np
from math import sin, radians, degrees, copysign
from pygame.math import Vector2

pygame 2.1.2 (SDL 2.0.18, Python 3.9.13)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [3]:
'''
class cible:
    trajecory = []
    def __init__(self, speed_0, position_0, acceleration_0):
        self.speed = speed_0
        self.position = position_0
        self.acceleration = acceleration_0
    def trajectoire(self):
        print(self.trajectory)
'''

'\nclass cible:\n    trajecory = []\n    def __init__(self, speed_0, position_0, acceleration_0):\n        self.speed = speed_0\n        self.position = position_0\n        self.acceleration = acceleration_0\n    def trajectoire(self):\n        print(self.trajectory)\n'

In [4]:
del_t = 1
k_x = 0.0001
k_v = 0.00001
k_col = 0.1
EPSILON = 10
N_drones = 20

In [5]:
class drone:
    trajectory = []
    def __init__(self, position_0, altitude_0=0.0, angle_0=0.0, speed_0=np.zeros(2), acceleration_0=np.zeros(2)):
        self.angle = angle_0
        self.speed = speed_0
        self.position = position_0
        self.acceleration = acceleration_0
        self.altitude = altitude_0
        self.max_speed_value = 0.5
        self.min_speed_value = 0.1
        #(120, 30, 190)
        self.color = (250, 0, 0)
        self.rect = pygame.Rect(self.position[0], self.position[1], 10, 10)
        self.max_speed = np.array([self.max_speed_value, self.max_speed_value])
        self.min_speed = np.array([self.min_speed_value, self.min_speed_value])

    def radius(self):
        return 10
    
    def draw(self, win):
        pygame.draw.circle(win, self.color, self.position, 1)

    def update(self, target_position, target_speed, drone_positions, N_drones):
        old_pos = self.position
        del_x = target_position - self.position
        del_v = target_speed - self.speed
        self.acceleration = k_x*del_x + k_v*del_v
        for i in range(N_drones):
            if(np.linalg.norm(self.position - drone_positions[i]) < EPSILON and np.linalg.norm(self.position - drone_positions[i])>0):
                self.acceleration += (k_col/np.linalg.norm(self.position - drone_positions[i])**2)*(self.position - drone_positions[i])
        
        self.speed = self.speed + self.acceleration*del_t

        self.speed = np.maximum(-self.max_speed, np.minimum(self.max_speed, self.speed))
        #self.speed = self.speed + np.random.normal(loc=0.0, scale=0.1, size=2)
        
        self.position = self.position + self.speed*del_t + 0.5*del_t**2*self.acceleration

        #self.orientation = (self.position - old_pos)/np.linalg.norm(self.position - old_pos)

        self.rect = pygame.Rect(int(self.position[0]), int(self.position[1]), 32, 32)

In [6]:
#Player Class
class Player:
    def __init__(self, x, y):
        self.position = np.array([x, y])
        self.rect = pygame.Rect(self.position[0], self.position[1], 20, 32)
        self.color = (250, 120, 60)
        self.speed = np.array([0, 0])
        self.left_pressed = False
        self.right_pressed = False
        self.up_pressed = False
        self.down_pressed = False
        self.pace = 1
    
    def draw(self, win):
        pygame.draw.rect(win, self.color, self.rect)
    
    def update(self):
        self.speed = np.array([0, 0])
        if self.left_pressed and not self.right_pressed:
            self.speed[0] = -self.pace
        if self.right_pressed and not self.left_pressed:
            self.speed[0] = self.pace
        if self.up_pressed and not self.down_pressed:
            self.speed[1] = -self.pace
        if self.down_pressed and not self.up_pressed:
            self.speed[1] = self.pace

        self.position[0] += self.speed[0]*del_t
        self.position[1] += self.speed[1]*del_t

        self.rect = pygame.Rect(int(self.position[0]), int(self.position[1]), 20, 32)

In [7]:
class swarm():
    def __init__(self, N_drones_0, launch_position_0):
        self.swarm = []
        self.drone_positions = []
        self.N_drones = N_drones_0
        for i in range(self.N_drones):
            self.swarm.append(drone(position_0=launch_position_0 + np.random.normal(loc=0, scale=100, size=2), speed_0=np.zeros(2), acceleration_0=np.zeros(2)))
            self.drone_positions.append(self.swarm[i].position)

    def update(self, target_position, target_speed):
        for i in range(self.N_drones):
            self.swarm[i].update(target_position, target_speed, self.drone_positions, self.N_drones)
            self.drone_positions[i] = self.swarm[i].position

    def draw(self, win):
        for i in range(self.N_drones):
            self.swarm[i].draw(win)

In [ ]:
class Car:
    def __init__(self, x, y, angle=0.0, length=4, max_steering=30, max_acceleration=5.0, scaling=1):
        self.position = Vector2(x, y)
        self.velocity = Vector2(0.0, 0.0)
        self.angle = angle
        self.length = length
        self.max_acceleration = max_acceleration
        self.max_steering = max_steering
        self.max_velocity = 20/scaling
        self.brake_deceleration = 10
        self.free_deceleration = 2

        self.acceleration = 0.0
        self.steering = 0.0

    def update(self, dt):
        self.velocity += (self.acceleration * dt, 0)
        self.velocity.x = max(-self.max_velocity, min(self.velocity.x, self.max_velocity))

        if self.steering:
            turning_radius = self.length / sin(radians(self.steering))
            angular_velocity = self.velocity.x / turning_radius
        else:
            angular_velocity = 0

        self.position += self.velocity.rotate(-self.angle) * dt
        self.angle += degrees(angular_velocity) * dt

In [ ]:
class Game:
    def __init__(self):
        pygame.init()
        pygame.display.set_caption("Drone pursuit")
        width = 1280
        height = 720
        self.screen = pygame.display.set_mode((width, height))
        self.clock = pygame.time.Clock()
        self.ticks = 60
        self.exit = False

    def run(self):
        current_dir = os.path.dirname(os.path.abspath(__file__))
        image_path = os.path.join(current_dir, "car.png")
        car_image = pygame.image.load(image_path)
        taille = car_image.get_size()
        scaling = 10
        taille = [taille[0]/scaling, taille[1]/scaling]
        car_image = pygame.transform.scale(car_image, taille)
        car = Car(0, 0, scaling=scaling)
        ppu = 32

        while not self.exit:
            dt = self.clock.get_time() / 1000

            # Event queue
            for event in pygame.event.get():
                if event.type == pygame.QUIT:
                    self.exit = True

            # User input
            pressed = pygame.key.get_pressed()

            if pressed[pygame.K_UP]:
                if car.velocity.x < 0:
                    car.acceleration = car.brake_deceleration
                else:
                    car.acceleration += (1/scaling) * dt
            elif pressed[pygame.K_DOWN]:
                if car.velocity.x > 0:
                    car.acceleration = -car.brake_deceleration
                else:
                    car.acceleration -= (1/scaling) * dt
            elif pressed[pygame.K_SPACE]:
                if abs(car.velocity.x) > dt * car.brake_deceleration:
                    car.acceleration = -copysign(car.brake_deceleration, car.velocity.x)
                else:
                    car.acceleration = -car.velocity.x / dt
            else:
                if abs(car.velocity.x) > dt * car.free_deceleration:
                    car.acceleration = -copysign(car.free_deceleration, car.velocity.x)
                else:
                    if dt != 0:
                        car.acceleration = -car.velocity.x / dt
            car.acceleration = max(-car.max_acceleration, min(car.acceleration, car.max_acceleration))

            if pressed[pygame.K_RIGHT]:
                car.steering -= 30 * dt
            elif pressed[pygame.K_LEFT]:
                car.steering += 30 * dt
            else:
                car.steering = 0
            car.steering = max(-car.max_steering, min(car.steering, car.max_steering))

            # Logic
            car.update(dt)

            # Drawing
            self.screen.fill((0, 0, 0))
            rotated_car = pygame.transform.rotate(car_image, car.angle)
            rect = rotated_car.get_rect()
            self.screen.blit(rotated_car, car.position * ppu - (rect.width / 2, rect.height / 2))
            pygame.display.flip()

            self.clock.tick(self.ticks)
        pygame.quit()

In [8]:
"""Pygame window initialisation"""
#Constants
WIDTH, HEIGHT = 1200, 800
TITLE = "Drone pursuit"

#pygame initialization
pygame.init()
win = pygame.display.set_mode((WIDTH, HEIGHT))
pygame.display.set_caption(TITLE)
clock = pygame.time.Clock()

##################################################
#Object initialisation
player = Player(WIDTH/2, HEIGHT/2)
swarm = swarm(N_drones, np.array([WIDTH/2, HEIGHT/2]))
##################################################

#Main Loop
while True:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
        if event.type == pygame.KEYDOWN:
            if event.key == pygame.K_LEFT:
                player.left_pressed = True
            if event.key == pygame.K_RIGHT:
                player.right_pressed = True
            if event.key == pygame.K_UP:
                player.up_pressed = True
            if event.key == pygame.K_DOWN:
                player.down_pressed = True
        if event.type == pygame.KEYUP:
            if event.key == pygame.K_LEFT:
                player.left_pressed = False
            if event.key == pygame.K_RIGHT:
                player.right_pressed = False
            if event.key == pygame.K_UP:
                player.up_pressed = False
            if event.key == pygame.K_DOWN:
                player.down_pressed = False
        
    #Draw
    win.fill((0,0,0))  
    player.draw(win)
    swarm.draw(win)
    font = pygame.font.SysFont("Verdana", 20)
    text = font.render(str(int(clock.get_fps())), True, (255,255,255))
    win.blit(text, (0,0))

    #update
    player.update()
    swarm.update(player.position, player.speed)
    pygame.display.flip()
    

    clock.tick(200)

KeyboardInterrupt: 

In [ ]:
import pygame
import math

def rotate_triangle(center, scale, mouse_pos):

    vMouse  = pygame.math.Vector2(mouse_pos)
    vCenter = pygame.math.Vector2(center)
    angle   = pygame.math.Vector2().angle_to(vMouse - vCenter)

    points = [(-0.5, -0.866), (-0.5, 0.866), (2.0, 0.0)]
    rotated_point = [pygame.math.Vector2(p).rotate(angle) for p in points]

    triangle_points = [(vCenter + p*scale) for p in rotated_point]
    return triangle_points

disp=pygame.display.set_mode((200,200))

run = True
while run:

    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            run = False

    mouse_position = pygame.mouse.get_pos()
    points = rotate_triangle((100, 100), 10, mouse_position)

    pygame.Surface.fill(disp, (255,255,255))
    pygame.draw.polygon(disp, (0,0,0), points)
    pygame.display.update()